In [16]:
import pandas as pd
import numpy as np
from datetime import datetime


import nltk
nltk.download('words')
from nltk.corpus import words

#NOTE, I copied this code from a stackexchange page. but I modified it also to remove duplicates from the output and to compute average size of words
list_of_words = list(filter(lambda x: len(x)>2, words.words()))  #filtering out single letter alphabets, since all passwords would have at least one alphabet 
def isWordinPassword(password):
    #accepts string password and returns list of words that exist in the password
    words_in_password = list(filter(lambda x: x in password, list_of_words))
    words_without_duplicates = []
    for x in words_in_password:
        if x not in words_without_duplicates:
            words_without_duplicates.append(x)
    lw = list(map(lambda x: len(x), words_without_duplicates))
    ave = sum(lw)/len(lw) if len(lw)>0 else 0
    return len(words_without_duplicates), ave

#took this from geeksforgeeks.org
def isPalindrome(s):
    return s == s[::-1]

def countTypes(strin):
    timesEachCharWasSeen = {strin[0]:1}
    
    a = b = c = d = 0
    a2 = b2 = c2 = d2 = 0
    a3 = b3 = c3 = d3 = 0
    flag = 0 #flag represents type of prior letter, 0, 1, 2, 3 for uppercase, lowercase, number, symbol
    if ord(strin[0]) in range(65, 91):
        a +=1
        a2 += 1
        flag = 0
    elif ord(strin[0]) in range(97, 123):
        b += 1
        b2 += 1
        flag = 1
    elif ord(strin[0]) in range(48, 58):
        c += 1
        c2 += 1
        flag = 2
    else:
        d += 1
        d2 += 1
        flag = 3
    
    for y in range(1, len(strin), 1):
        if strin[y] not in timesEachCharWasSeen:
            timesEachCharWasSeen[strin[y]] = 1
        else:
            timesEachCharWasSeen[strin[y]] += 1
        z = ord(strin[y])

        if z in range(65, 91):
            a+=1
            if flag == 0:
                a2+=1
            else:
                if a2 > a3:
                    a3 = a2
                a2 = 1
                flag = 0
        elif z in range(97, 123):
            b+=1
            if flag == 1:
                b2+=1
            else:
                if b2 > b3:
                    b3 = b2
                b2 = 1
                flag = 1
        elif z in range(48, 58):
            c+=1
            if flag == 2:
                c2+=1
            else:
                if c2 > c3:
                    c3 = c2
                c2 = 1
                flag = 2
        else:
            d+=1
            if flag == 3:
                d2+=1
            else:
                if d2 > d3:
                    d3 = d2
                d2 = 1
                flag = 3
    if a2 > a3:
        a3 = a2
    if b2 > b3:
        b3 = b2
    if c2 > c3:
        c3 = c2
    if d2 > d3:
        d3 = d2
    
    
    k = max(timesEachCharWasSeen, key=timesEachCharWasSeen.get)
    return a, a3, b, b3, c, c3, d, d3, k 

def analyzeString(strin):
    a, a2, b, b2, c, c2, d, d2, k= countTypes(strin)
    #a, b, c, d represent the number of uppercase letters, lowercase letters, numbers, and symbols
    #a2, b2, c2, d2 represent the size of the largest continuous group of uppercase, lowercase, numbers, and symbols
    
    
    #e represents the type of the first character in the string. it is one-hot encoded like this [uppercase, lowercase, number, symbol]
    if ord(strin[0]) in range(65, 91):
        e = [1, 0, 0, 0]
    elif ord(strin[0]) in range(97, 123):
        e = [0, 1, 0, 0]
    elif ord(strin[0]) in range(48, 58):
        e = [0, 0, 1, 0]
    else:
        e = [0, 0, 0, 1]
    #f represents the type of the last character, one-hot encoded like this [uppercase, lowercase, number, symbol]
    if ord(strin[-1]) in range(65, 91):
        f = [1, 0, 0, 0]
    elif ord(strin[-1]) in range(97, 123):
        f = [0, 1, 0, 0]
    elif ord(strin[-1]) in range(48, 58):
        f = [0, 0, 1, 0]
    else:
        f = [0, 0, 0, 1]
    
    g, h = isWordinPassword(strin) #g is number of english words in the password, h is average length of those words
    
    return a, a2, b, b2, c, c2, d, d2, e, f, g, h, int(isPalindrome(strin))#, k

data = pd.read_csv("data.tsv", index_col=False, usecols=[0, 1], sep='\t')

[nltk_data] Downloading package words to /home/pzoura/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [17]:
print(len(data))

25000


In [19]:
data2 = data.values

dict = {'Password':[],
        'Strength':[],
        'Length':[],
        '#Uppers':[],
        '#Lowers':[],
        '#Numbers':[],
        '#Symbols':[],
        'SLG_uppers':[], #size of largest group of uppercase letters
        'SLG_lowers':[],
        'SLG_numbers':[],
        'SLG_symbols':[],
        'FCT_Up':[],
        'FCT_Lo':[],
        'FCT_Nu':[],
        'FCT_Sy':[],
        'LCT_Up':[],
        'LCT_Lo':[],
        'LCT_Nu':[],
        'LCT_Sy':[],
        '#words':[],
        'ave_size_of_words':[],
        'palindrome':[]
        #'most_common_char':[]
       }

df = pd.DataFrame(dict)

for i in range(len(data)):
                                            #, k = analyzeString(data2[i][0])
    a, a2, b, b2, c, c2, d, d2, e, f, g, h, j = analyzeString(data2[i][0])
    df.loc[i] = [data2[i][0], data2[i][1], len(data2[i][0]), a, b, c, d, a2, b2, c2, d2, e[0], e[1], e[2], e[3], f[0], f[1], f[2], f[3], g, h, j]
    if i%1000==0:
        print(i)
df.to_csv("data2.tsv", sep='\t')

#now I just need to make a function to calculate each password part.

0
